# Search Tree Performance Analysis
Because why not

In [ ]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

from game import GameState
from tree_ai import GameTree

## Performance Data Collection
This might take some time

In [ ]:
time_freq = 20 # defines the time granularity

trials = 20
timeout = 1000 # After a depth takes this many seconds on average to compute, data collection will stop

columns = ['depth', 'leafs', 'time', 'rate']

output_file = 'ai_perf.csv'

In [ ]:
def run_trial(depth) -> np.ndarray:
    t = GameTree(GameState())
    
    start = time.time_ns()
    while t.get_depth() <= depth: 
        time.sleep(1/time_freq)
    end = time.time_ns()
    
    t.kill_thread()
    
    depth = t.get_depth()-1 # minus 1 because this returns max depth, and we want most complete depth
    leafs = t.get_leaves().size
    sec   = round((end-start) * 10 ** -9, 3)
    rate  = leafs/sec
    
    return np.array([depth, leafs, sec, rate])

In [ ]:
# Setup output file
with open(output_file, mode='w') as f:
    f.write(','.join(columns)+'\n')

# Run each trial and immediately write result to csv
depth, avg_timeout = 2, 0
while avg_timeout < timeout:
    avg_timeout = 0
    for trial in range(trials):
        print(f'Running trial {trial+1}/{trials} @ target depth {depth}', end=' ... ')
        
        row = [run_trial(depth)]
        df = pd.DataFrame([run_trial(depth)], columns=columns)
        df.to_csv(output_file, mode='a', header=False, index=False)
        
        trial_time = df['time'][0]
        avg_timeout += trial_time
        print(f'finished in {trial_time} s')
        
    avg_timeout /= trials
    depth += 1

In [ ]:
df = pd.DataFrame(np.array(data), columns=columns)
df.to_csv(output_file, mode='a')

df.groupby('depth').mean()

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
sn.boxplot(x=df['depth'], y=df['leafs'], ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
sn.boxplot(x=df['depth'], y=df['time'], ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
sn.boxplot(x=df['depth'], y=df['rate'], ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
sn.scatterplot(x=df['leafs'], y=df['time'], hue='depth', palette="Set1", data=df, ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
sn.regplot(x=df['leafs'], y=df['time'], data=df, ax=ax)